In [51]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Normalization, Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy, MeanSquaredError

use_coordination = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
dataset = pd.read_csv("datasets/coords_dataset.csv")
target = [0, 1]
input_dataset = []
labels = ['hello', 'happy']
initdata = []

def setting_dataset(dataset, labels):
    for label in labels:
        return_dataset = dataset[dataset['class'] == label]
        return_dataset = return_dataset.drop(labels='class', axis=1)
        initdata.append(return_dataset)  
    return np.array(initdata, np.float64)

# data_hello = setting_dataset(dataset, 'hello') 
# data_happy = setting_dataset(dataset, 'happy')
data = setting_dataset(dataset, labels)
input_dataset = np.concatenate([data_hello, data_happy], axis=0)
input_dataset = np.array(input_dataset)

(2, 60, 36)

In [52]:
loss = BinaryCrossentropy()

model = Sequential([
    LSTM(300, activation='relu', input_shape=(input_dataset.shape[1:])),
    Dense(200, activation = 'relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(1, activation='softmax')
])

model.compile(optimizer=Adam(),
              loss='BinaryCrossentropy',
              metrics=['acc'])

EPOCHS = 200
BATCHSIZE = 1

model.fit(input_dataset, target, epochs=EPOCHS, batch_size=BATCHSIZE)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

In [49]:
y_pred = model.predict(input_dataset)
y_pred

1/1 [==============================] - 0s 181ms/step


array([[1.],
       [1.]], dtype=float32)